version con graficos

In [ ]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt

def graFunction(rangX,alto,bajo,medio,nameGraf,labelAlto,labelBajo,labelMedio,):
    plt.plot(rangX, alto, 'b', linewidth=1.5, label=labelAlto)
    plt.plot(rangX, medio, 'g', linewidth=1.5, label=labelMedio)
    plt.plot(rangX, bajo, 'r', linewidth=1.5, label=labelBajo)
    plt.title(nameGraf)
    plt.legend()
    plt.show()

# Definición del rango de las variables
tempHabitacion = np.arange(5, 36, 1)  
tempCaldera = np.arange(20, 100, 1)   
combustion = np.arange(1, 101, 1) 


# Funciones de membresía para la temperatura exterior(hab)
# Baja (frío):
#tempHabitacion_baja = fuzz.gbellmf(tempHabitacion, 5,2,10)
tempHabitacion_baja = fuzz.trimf(tempHabitacion, [5, 5, 17])
# Media (templado): 
tempHabitacion_media = fuzz.gbellmf(tempHabitacion, 8,2,20)
# Alta (caluroso): 
#tempHabitacion_alta = fuzz.gbellmf(tempHabitacion,5,2,30)
tempHabitacion_alta = fuzz.trimf(tempHabitacion, [23, 35, 35])

# Funciones de membresía para la temperatura interior(cal)
tempCaldera_normal = fuzz.gbellmf(tempCaldera, 20, 3, 20)
tempCaldera_alta = fuzz.gbellmf(tempCaldera, 22, 3, 60)
tempCaldera_critica = fuzz.gbellmf(tempCaldera, 20, 2, 100)

# Funciones de membresía para la combustión
# Combustión alta: mayor intensidad de la llama, entre 50 y 100%
combustion_alta = fuzz.trimf(combustion, [50, 100, 100])
# Combustión moderada: intensidad moderada de la llama, entre 30 y 70%
combustion_moderada = fuzz.trimf(combustion, [20, 50, 80])
# Combustión en piloto: llama mínima o apagada, entre 0 y 30%
combustion_piloto = fuzz.trimf(combustion, [0, 0, 30])

#graficos ABM

graFunction(tempHabitacion,tempHabitacion_alta,tempHabitacion_baja,tempHabitacion_media,
                'temperatura externa(habitacion)','temperatura alta','temperatura baja','temperatura media')
graFunction(tempCaldera,tempCaldera_critica,tempCaldera_normal,tempCaldera_alta,
            'temperatura interna(caldera)','Tcaldera critica','Tcaldera normal','Tcaldera alta')
graFunction(combustion,combustion_alta,combustion_piloto,combustion_moderada,
            '% llama','-> maximo','->piloto','->moderado ')


# Valores de entrada
tempActual_habitacion = 28   # Cambia este valor para probar
tempActual_caldera = 90   # Cambia este valor para probar


# Fuzzificación de las entradas
temp_ext_level_low = fuzz.interp_membership(tempHabitacion, tempHabitacion_baja, tempActual_habitacion)
temp_ext_level_med = fuzz.interp_membership(tempHabitacion, tempHabitacion_media, tempActual_habitacion)
temp_ext_level_high = fuzz.interp_membership(tempHabitacion, tempHabitacion_alta, tempActual_habitacion)

temp_int_level_normal = fuzz.interp_membership(tempCaldera, tempCaldera_normal, tempActual_caldera)
temp_int_level_high = fuzz.interp_membership(tempCaldera, tempCaldera_alta, tempActual_caldera)
temp_int_level_critical = fuzz.interp_membership(tempCaldera, tempCaldera_critica, tempActual_caldera)

# Aplicación de las reglas
# 1. Cuando la temperatura exterior es baja, la combustión debe ser alta
reglaActivacion_1 = np.fmin(temp_ext_level_low, combustion_alta)

# 2. Cuando la temperatura exterior es media, la combustión debe ser moderada
reglaActivacion_2 = np.fmin(temp_ext_level_med, combustion_moderada)

# 3. Cuando la temperatura exterior es alta, la combustión debe estar en piloto
reglaActivacion_3 = np.fmin(temp_ext_level_high, combustion_piloto)

# 4. Cuando la temperatura interior sea alta, la combustión debe limitarse ligeramente (se reduce un poco)
reglaActivacion_4 = np.fmin(temp_int_level_high, combustion_moderada)

# 5. Cuando la temperatura interior sea crítica, la combustión debe reducirse mucho
reglaActivacion_5 = np.fmin(temp_int_level_critical, combustion_piloto)

# Agregación de todas las reglas
aggregated = np.fmax(reglaActivacion_1, np.fmax(reglaActivacion_2, np.fmax(reglaActivacion_3, np.fmax(reglaActivacion_4, reglaActivacion_5))))

# Defuzzificación - cálculo del centroide
combustion_output = fuzz.defuzz(combustion, aggregated, 'centroid')

# Resultado de la combustión
print(f"El tamaño de la llama de combustión recomendado es: {combustion_output:.2f}%")

# Visualización
plt.plot(combustion, combustion_alta, 'b', linewidth=1.5, label='Combustión Alta')
plt.plot(combustion, combustion_moderada, 'g', linewidth=1.5, label='Combustión Moderada')
plt.plot(combustion, combustion_piloto, 'r', linewidth=1.5, label='Combustión Piloto')
#plt.fill_between(combustion, 0, aggregated, color='Orange', alpha=0.7)
plt.title('Resultado de Agregación de Reglas y Defuzzificación')
plt.legend()
plt.show() 

version que genera calores.txt para ver todas las combinaciones

In [ ]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt

# Definición del rango de las variables
tempHabitacion = np.arange(5, 36, 1)  
tempCaldera = np.arange(20, 100, 1)   
combustion = np.arange(1, 101, 1) 

# Funciones de membresía para la temperatura exterior(hab)
# Baja (frío):
#tempHabitacion_baja = fuzz.gbellmf(tempHabitacion, 5,2,10)
tempHabitacion_baja = fuzz.trimf(tempHabitacion, [5, 5, 17])
# Media (templado): 
tempHabitacion_media = fuzz.gbellmf(tempHabitacion, 8,2,20)
# Alta (caluroso): 
#tempHabitacion_alta = fuzz.gbellmf(tempHabitacion,5,2,30)
tempHabitacion_alta = fuzz.trimf(tempHabitacion, [23, 35, 35])

# Funciones de membresía para la temperatura interior(cal)
tempCaldera_normal = fuzz.gbellmf(tempCaldera, 20, 3, 20)
tempCaldera_alta = fuzz.gbellmf(tempCaldera, 22, 3, 60)
tempCaldera_critica = fuzz.gbellmf(tempCaldera, 20, 2, 100)

combustion_alta = fuzz.trimf(combustion, [50, 100, 100])
combustion_moderada = fuzz.trimf(combustion, [20, 50, 80])
combustion_piloto = fuzz.trimf(combustion, [0, 0, 30])


for i in range(5,36):
    for j in range(20,100):
        tempActual_habitacion = i   # Cambia este valor para probar
        tempActual_caldera = j 
        # Fuzzificación de las entradas
        temp_ext_level_low = fuzz.interp_membership(tempHabitacion, tempHabitacion_baja, tempActual_habitacion)
        temp_ext_level_med = fuzz.interp_membership(tempHabitacion, tempHabitacion_media, tempActual_habitacion)
        temp_ext_level_high = fuzz.interp_membership(tempHabitacion, tempHabitacion_alta, tempActual_habitacion)

        temp_int_level_normal = fuzz.interp_membership(tempCaldera, tempCaldera_normal, tempActual_caldera)
        temp_int_level_high = fuzz.interp_membership(tempCaldera, tempCaldera_alta, tempActual_caldera)
        temp_int_level_critical = fuzz.interp_membership(tempCaldera, tempCaldera_critica, tempActual_caldera)

        # Aplicación de las reglas
        # 1. Cuando la temperatura exterior es baja, la combustión debe ser alta
        reglaActivacion_1 = np.fmin(temp_ext_level_low, combustion_alta)
        # 2. Cuando la temperatura exterior es media, la combustión debe ser moderada
        reglaActivacion_2 = np.fmin(temp_ext_level_med, combustion_moderada)
        # 3. Cuando la temperatura exterior es alta, la combustión debe estar en piloto
        reglaActivacion_3 = np.fmin(temp_ext_level_high, combustion_piloto)
        # 4. Cuando la temperatura interior sea alta, la combustión debe limitarse ligeramente (se reduce un poco)
        reglaActivacion_4 = np.fmin(temp_int_level_high, combustion_moderada)
        # 5. Cuando la temperatura interior sea crítica, la combustión debe reducirse mucho
        reglaActivacion_5 = np.fmin(temp_int_level_critical, combustion_piloto)
        # Agregación de todas las reglas
        aggregated = np.fmax(reglaActivacion_1, np.fmax(reglaActivacion_2, np.fmax(reglaActivacion_3, np.fmax(reglaActivacion_4, reglaActivacion_5))))
        # Defuzzificación - cálculo del centroide
        combustion_output = fuzz.defuzz(combustion, aggregated, 'centroid')

        with open('valores.txt', 'a') as file:
            file.write(f" temp habitacion : {i} temp caldera: {tempActual_caldera} % llama: {combustion_output} \n")